In [1]:
using TextAnalysis, CorpusLoaders, MultiResolutionIterators

┌ Info: Recompiling stale cache file /home/ayushk4/.julia/compiled/v1.0/TextAnalysis/5Mwet.ji for TextAnalysis [a2db99b7-8b79-58f8-94bf-bbc811eef33d]
└ @ Base loading.jl:1190
┌ Warning: Package TextAnalysis does not have Libdl in its dependencies:
│ - If you have TextAnalysis checked out for development and have
│   added Libdl as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with TextAnalysis
└ Loading Libdl into TextAnalysis from project dependency, future warnings for TextAnalysis are suppressed.
┌ Info: Recompiling stale cache file /home/ayushk4/.julia/compiled/v1.0/CorpusLoaders/jEgYN.ji for CorpusLoaders [8f4694b6-a17a-549f-a57e-76e1fca3f017]
└ @ Base loading.jl:1190
┌ Warning: Package CorpusLoaders does not have CSV in its dependencies:
│ - If you have CorpusLoaders checked out for development and have
│   added CSV as a dependency but haven't updated your primary
│   environment'

In [2]:
pos = POS_Tagger()

TextAnalysis.POS_model{TextAnalysis.BiLSTM_CNN_CRF_Model{Flux.Conv{2,2,typeof(identity),TrackedArray{…,Array{Float32,4}},TrackedArray{…,Array{Float32,1}}},Array{Float32,2},Flux.Recur{Flux.LSTMCell{TrackedArray{…,Array{Float32,2}},TrackedArray{…,Array{Float32,1}}}},Flux.Dense{typeof(identity),TrackedArray{…,Array{Float32,2}},TrackedArray{…,Array{Float32,1}}},CRF{TrackedArray{…,Array{Float32,2}}},Array{Int64,2}}}(TextAnalysis.BiLSTM_CNN_CRF_Model{Flux.Conv{2,2,typeof(identity),TrackedArray{…,Array{Float32,4}},TrackedArray{…,Array{Float32,1}}},Array{Float32,2},Flux.Recur{Flux.LSTMCell{TrackedArray{…,Array{Float32,2}},TrackedArray{…,Array{Float32,1}}}},Flux.Dense{typeof(identity),TrackedArray{…,Array{Float32,2}},TrackedArray{…,Array{Float32,1}}},CRF{TrackedArray{…,Array{Float32,2}}},Array{Int64,2}}(["NNP", "VBZ", "JJ", "NN", "TO", "VB", ".", "CD", "DT", "VBD"  …  "NNPS", "SYM", "RBS", "UH", "PDT", "''", "LS", "JJS", "WP\$", "NN|SYM"], Dict('w'=>81,'E'=>34,'7'=>22,'Z'=>55,']'=>57,'o'=>73,'B

In [3]:
train_set = CorpusLoaders.load(CoNLL2000(), "train") # training set
test_set = CorpusLoaders.load(CoNLL2000(), "test") # test set

CorpusLoaders.Document{Array{Array{CorpusLoaders.POSTaggedWord,1},1},String}("test.txt", Array{CorpusLoaders.POSTaggedWord,1}[[POSTaggedWord("NNP", "B-NP", "Rockwell"), POSTaggedWord("NNP", "I-NP", "International"), POSTaggedWord("NNP", "I-NP", "Corp."), POSTaggedWord("POS", "B-NP", "'s"), POSTaggedWord("NNP", "I-NP", "Tulsa"), POSTaggedWord("NN", "I-NP", "unit"), POSTaggedWord("VBD", "B-VP", "said"), POSTaggedWord("PRP", "B-NP", "it"), POSTaggedWord("VBD", "B-VP", "signed"), POSTaggedWord("DT", "B-NP", "a")  …  POSTaggedWord("TO", "B-VP", "to"), POSTaggedWord("VB", "I-VP", "provide"), POSTaggedWord("JJ", "B-NP", "structural"), POSTaggedWord("NNS", "I-NP", "parts"), POSTaggedWord("IN", "B-PP", "for"), POSTaggedWord("NNP", "B-NP", "Boeing"), POSTaggedWord("POS", "B-NP", "'s"), POSTaggedWord("CD", "I-NP", "747"), POSTaggedWord("NNS", "I-NP", "jetliners"), POSTaggedWord(".", "O", ".")], [POSTaggedWord("NNP", "B-NP", "Rockwell"), POSTaggedWord("VBD", "B-VP", "said"), POSTaggedWord("DT", "B

In [9]:
function change_bracket(x)
    if x == "-LRB-" || x == "-LCB-"
        return "("
    elseif x == "-RRB-" || x == "-RCB-"
        return ")"
    end

    return x
end

function obtain_X_Y(dataset)
    X = [change_bracket.(CorpusLoaders.word.(sent)) for sent in dataset]
    Y = [CorpusLoaders.part_of_speech.(sent) for sent in dataset]
    return X, Y
end

X_train, Y_train = obtain_X_Y(train_set)
X_test, Y_test = obtain_X_Y(test_set)

(Array{String,1}[["Rockwell", "International", "Corp.", "'s", "Tulsa", "unit", "said", "it", "signed", "a"  …  "to", "provide", "structural", "parts", "for", "Boeing", "'s", "747", "jetliners", "."], ["Rockwell", "said", "the", "agreement", "calls", "for", "it", "to", "supply", "200", "additional", "so-called", "shipsets", "for", "the", "planes", "."], ["These", "include", ",", "among", "other", "parts", ",", "each", "jetliner", "'s"  …  "edges", "for", "the", "wings", "and", "an", "aft", "keel", "beam", "."], ["Under", "the", "existing", "contract", ",", "Rockwell", "said", ",", "it", "has", "already", "delivered", "793", "of", "the", "shipsets", "to", "Boeing", "."], ["Rockwell", ",", "based", "in", "El", "Segundo", ",", "Calif.", ",", "is", "an", "aerospace", ",", "electronics", ",", "automotive", "and", "graphics", "concern", "."], ["Frank", "Carlucci", "III", "was", "named", "to", "this", "telecommunications", "company", "'s"  …  "created", "by", "the", "death", "of", "William", "

In [10]:
function evaluate(pos, X, Y)
    num_correct = 0
    num_total = 0
    
    for (x_seq, y_seq) in zip(X, Y)
        preds = pos(x_seq)
    
        for i in eachindex(preds)
            num_total += 1
            if preds[i] == y_seq[i]
                num_correct += 1
            end
        end
    end

    println("Accuracy: ", num_correct / num_total)
end

evaluate (generic function with 1 method)

In [11]:
evaluate(pos, X_test, Y_test)

Accuracy: 0.9004580281571226


In [12]:
evaluate(pos, X_train, Y_train)

Accuracy: 0.9004614432736495
